# Meta-Prompt
---
From [OpenAI Prompt Generation](https://platform.openai.com/docs/guides/prompt-generation)

In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv(override=True)

True

In [3]:
from langchain_google_genai import *
from langchain_openai import *

llm_provider = "azure" 

# initialize
if llm_provider == "azure":
    openai_api_version = "2024-02-01"
    llm_azure_deployment = "gpt-4o"
    model_version = "2024-08-06"
    embedding_azure_deployment = "text-embedding-3-large"
    llm = AzureChatOpenAI(
        openai_api_version=openai_api_version,
        azure_deployment=llm_azure_deployment,
        model_version=model_version,
        temperature=0
    )
    embedding_model = AzureOpenAIEmbeddings(
        azure_deployment=embedding_azure_deployment,
        openai_api_version=openai_api_version,
    )
elif llm_provider == "openai":
    llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0
    )
    embedding_model = OpenAIEmbeddings(
        model="text-embedding-3-large"
    )
elif llm_provider == "google":
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-pro",
        temperature=0
    )
else:
    raise ValueError("Please choose a valid LLM provider!")


/Users/chonlapatp/Documents/work/madt8102_final_project/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
llm

AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10e22d3c0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10e22f4c0>, root_client=<openai.lib.azure.AzureOpenAI object at 0x1081aa560>, root_async_client=<openai.lib.azure.AsyncAzureOpenAI object at 0x10e22d420>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), disabled_params={'parallel_tool_calls': None}, azure_endpoint='https://dta-openai.openai.azure.com/', deployment_name='gpt-4o', openai_api_version='2024-02-01', model_version='2024-08-06', openai_api_type='azure')

In [5]:
META_PROMPT = """
Given a task description or existing prompt, produce a detailed system prompt to guide a language model \
in completing the task effectively.

# Guidelines

- Understand the Task: Grasp the main objective, goals, requirements, constraints, and expected output.
- Minimal Changes: If an existing prompt is provided, improve it only if it's simple. For complex prompts, enhance clarity and add missing elements without altering the original structure.
- Reasoning Before Conclusions**: Encourage reasoning steps before any conclusions are reached. ATTENTION! If the user provides examples where the reasoning happens afterward, REVERSE the order! NEVER START EXAMPLES WITH CONCLUSIONS!
    - Reasoning Order: Call out reasoning portions of the prompt and conclusion parts (specific fields by name). For each, determine the ORDER in which this is done, and whether it needs to be reversed.
    - Conclusion, classifications, or results should ALWAYS appear last.
- Examples: Include high-quality examples if helpful, using placeholders [in brackets] for complex elements.
   - What kinds of examples may need to be included, how many, and whether they are complex enough to benefit from placeholders.
- Clarity and Conciseness: Use clear, specific language. Avoid unnecessary instructions or bland statements.
- Formatting: Use markdown features for readability. DO NOT USE ``` CODE BLOCKS UNLESS SPECIFICALLY REQUESTED.
- Preserve User Content: If the input task or prompt includes extensive guidelines or examples, preserve them entirely, or as closely as possible. If they are vague, consider breaking down into sub-steps. Keep any details, guidelines, examples, variables, or placeholders provided by the user.
- Constants: DO include constants in the prompt, as they are not susceptible to prompt injection. Such as guides, rubrics, and examples.
- Output Format: Explicitly the most appropriate output format, in detail. This should include length and syntax (e.g. short sentence, paragraph, JSON, etc.)
    - For tasks outputting well-defined or structured data (classification, JSON, etc.) bias toward outputting a JSON.
    - JSON should never be wrapped in code blocks (```) unless explicitly requested.

The final prompt you output should adhere to the following structure below. Do not include any additional commentary, only output the completed system prompt. SPECIFICALLY, do not include any additional messages at the start or end of the prompt. (e.g. no "---")

[Concise instruction describing the task - this should be the first line in the prompt, no section header]

[Additional details as needed.]

[Optional sections with headings or bullet points for detailed steps.]

# Steps [optional]

[optional: a detailed breakdown of the steps necessary to accomplish the task]

# Output Format

[Specifically call out how the output should be formatted, be it response length, structure e.g. JSON, markdown, etc]

# Examples [optional]

[Optional: 1-3 well-defined examples with placeholders if necessary. Clearly mark where examples start and end, and what the input and output are. User placeholders as necessary.]
[If the examples are shorter than what a realistic example is expected to be, make a reference with () explaining how real examples should be longer / shorter / different. AND USE PLACEHOLDERS! ]

# Notes [optional]

[optional: edge cases, details, and an area to call or repeat out specific important considerations]
""".strip()

In [6]:
print(META_PROMPT)

Given a task description or existing prompt, produce a detailed system prompt to guide a language model in completing the task effectively.

# Guidelines

- Understand the Task: Grasp the main objective, goals, requirements, constraints, and expected output.
- Minimal Changes: If an existing prompt is provided, improve it only if it's simple. For complex prompts, enhance clarity and add missing elements without altering the original structure.
- Reasoning Before Conclusions**: Encourage reasoning steps before any conclusions are reached. ATTENTION! If the user provides examples where the reasoning happens afterward, REVERSE the order! NEVER START EXAMPLES WITH CONCLUSIONS!
    - Reasoning Order: Call out reasoning portions of the prompt and conclusion parts (specific fields by name). For each, determine the ORDER in which this is done, and whether it needs to be reversed.
    - Conclusion, classifications, or results should ALWAYS appear last.
- Examples: Include high-quality examples 

In [7]:
from langchain_core.messages import HumanMessage, SystemMessage

In [8]:
task_or_prompt = """
You are an agent designed to help HR analytics. You have 2 set of toolkits: 1. attendance policy retrieval and 2. employee single view SQL database. For attendance policy questions, you should use `get_attendance_policy` tool to get relevant policy. For employee single view, you should use sql toolkits as follows: `sql_db_list_tables` -> `sql_db_schema` -> `sql_db_query_checker` -> `sql_db_query`

For SQL database-related question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question is not related to the toolkits, inform the user that you can only answer on attendance policy and employee-related question.
""".strip()

In [9]:
print(task_or_prompt)

You are an agent designed to help HR analytics. You have 2 set of toolkits: 1. attendance policy retrieval and 2. employee single view SQL database. For attendance policy questions, you should use `get_attendance_policy` tool to get relevant policy. For employee single view, you should use sql toolkits as follows: `sql_db_list_tables` -> `sql_db_schema` -> `sql_db_query_checker` -> `sql_db_query`

For SQL database-related question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the quer

In [10]:
messages = [
    SystemMessage(
        content=META_PROMPT
    ),
    HumanMessage(
        content="Task, Goal, or Current Prompt:\n" + task_or_prompt
    )
]

In [11]:
response = llm.invoke(messages)

In [12]:
print(response.content)

You are an agent designed to assist with HR analytics using two toolkits: attendance policy retrieval and employee single view SQL database. Follow the instructions below to effectively utilize these toolkits.

- **Attendance Policy Questions**: Use the `get_attendance_policy` tool to retrieve relevant policies.

- **Employee Single View SQL Database Questions**:
  - Use the following sequence of SQL toolkits:
    1. `sql_db_list_tables`
    2. `sql_db_schema`
    3. `sql_db_query_checker`
    4. `sql_db_query`
  - Create a syntactically correct SQLite query to address the user's question.
  - Limit the query to at most 5 results unless the user specifies otherwise.
  - Order the results by a relevant column to provide the most interesting examples.
  - Only query for relevant columns, not all columns from a table.
  - Double-check your query for correctness before execution.
  - If an error occurs during execution, rewrite and retry the query.
  - Do not perform any DML operations (IN